In [ ]:
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(model="gpt-4o-mini") 

response = chat.invoke("한국에 도시가 몇개인가요??")

print(response)


content='한국에는 2023년 기준으로 17개의 광역자치단체가 있으며, 이 중 6개는 특별시와 광역시입니다. 나머지 11개는 도(道)입니다. 각 도와 광역시 아래에는 여러 개의 시와 군이 있습니다. 전체적으로 한국의 도시 수는 수백 개에 달합니다. 정확한 수치는 행정 구역의 변화에 따라 달라질 수 있습니다.' response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 16, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None} id='run-8fdc22b4-ee4b-464e-8cfc-a3e8d8f8f53f-0'


In [ ]:
# 🟢 1. 일반 텍스트 프롬프트 템플릿 

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)


#지금은 세줄이지만 이것도 한줄로 바뀔것입니다..

template = PromptTemplate.from_template(
    "{country_a}와 {country_b}의 거리는 얼마인가요??"
)

formatted_prompt = template.format(country_a="한국", country_b="대만")

response1 = chat.invoke(formatted_prompt)

print("🟢 Response 1:", response1.content)



🟢 Response 1: 한국과 대만의 거리는 대략 1,600킬로미터 정도입니다. 서울에서 타이베이까지의 직선 거리로 계산했을 때의 대략적인 수치이며, 실제 비행 거리나 항로에 따라 다소 차이가 있을 수 있습니다.


In [7]:
# 🟢 2. 역할 기반 채팅 프롬프트 템플릿

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv


chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?"),
])

chat_messages = chat_template.format_messages(
    language="한국어",
    name="키움증권",
    country_a="Mexico",
    country_b="Thailand",
)

response2 = chat.invoke(chat_messages)

print("🟢 Response 2:", response2.content)


🟢 Response 2: 멕시코와 태국 사이의 거리는 약 14,000킬로미터 정도입니다. 제 이름은 키움증권입니다! 다른 질문이 있으신가요?


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import BaseOutputParser
from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))  # 각 항목의 공백 제거 후 리스트로 반환

# ✅ 대화 프롬프트 템플릿 정의 (역할 기반)
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

# ✅ 체인을 연결 (프롬프트 → 모델 → 파서)
chain = template | chat | CommaOutputParser()

# ✅ 실행 (프롬프트에 값 넣고 체인 작동)
response = chain.invoke({
    "max_items": 5,
    "question": "우주행성에는 어떤것이있지?"
})

# ✅ 결과 출력
print(response)


['지구', '화성', '목성', '금성', '토성']


In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import BaseOutputParser
from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# ✅ 출력 파서를 만들어 콤마로 구분된 문자열을 리스트로 변환
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))  # 각 항목의 공백 제거 후 리스트로 반환

# ✅ 대화 프롬프트 템플릿 정의 (역할 기반)→ AI에게 역할을 줘요.
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])


# ✅ 체인을 연결 (프롬프트 → 모델 → 파서) 
chain = template | chat | CommaOutputParser()

response = chain.invoke({
    "max_items": 5,
    "question": "세상에서 가장 비싼 음식은?"
})

# ✅ 결과 출력
print(response)


['트뤼플', '카비어', '사프란', '와규', '블랙 가리비']
